<a href="https://colab.research.google.com/github/ThapasPPramod/100DaysOfCode/blob/main/Blackjack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pandas append() is deprecated; remember to modify it to concat() in the future

In [ ]:
from abc import ABC, abstractmethod
import random, pandas as pd
from IPython.display import clear_output

In [ ]:
class BrokeException(Exception):
  def __init__(self):
    message = "You are broke! Get out of this game !"
    super.__init__(message)
    return message

In [ ]:
class Blackjack(ABC):
  def __init__(self):
    self.db = self.getdb()
    self.prompts = {"login": ('Enter your LoginId : ', 'Enter your Password : ', 'Existing user?(y/n)'),
                    "reg": ('Create LoginId : ', 'Create Password : '),
                    "ace":'Choose value for your Ace card(1 or 11) : ',
                    "game": ('How much do you wanna bet? : ','Stand, Hit?(s/h) :','Stand, Hit or Double?(s/h/d)')
                    }
    self.session, self.quit = self.create_session()
    self.log_tries = 0
    while self.quit:
      if self.log_tries > 3:
        print('You have made too many attempts! ')
        return False
      self.session,self.quit = self.create_session()
      self.log_tries += 1
    self.target = 21
    return True
  def getdb(self):
    try:
      df = pd.read_csv('bjdata.csv')
    except Exception as e:
      df = pd.DataFrame(columns=['loginId','Passwd','Credit'])
    return df
  @abstractmethod
  def getHand(self):
    pass
  def checkDb(self, data)->bool:
    try:
      # print(data, self.db)
      person = self.db.loc[(self.db['loginId'] == data[0]) & (self.db['Passwd'] == data[1])]
      clear_output(wait=True)
      print('Hi ',data[0])
      if not person.empty:
        self.credit = int(person['Credit'])
        print('Your credit: ', self.credit)
        return True
    except Exception as e:
      print("Login Credentials do not Match!", e)
      return False
  def updateDb(self, data)->bool:
    try:
      new = {'loginId':data[0], 'Passwd':data[1], 'Credit':1000}
      # print(new)
      self.db = self.db.append(new, ignore_index=True)
      self.db.to_csv('./bjdata.csv', index=False)
      # print('db: ', self.db)
    except Exception as e:
      print('error : ', e)
      return False
    return True
  def updateCredit(self):
    # updated = {'loginId':self.loginid,'Credit':self.credit}
    self.db.loc[self.db['loginId'] == self.loginid,'Credit'] = int(self.credit)
  def upload_to_db(self):
    self.db.to_csv('./bjdata.csv', index=False)
  def create_session(self):
    clear_output(wait=True)
    if (input(self.prompts['login'][2])) == 'y':
      login_cred = [input(self.prompts['login'][0]), input(self.prompts['login'][1]), False]
    else:
      login_cred = [input(self.prompts['reg'][0]), input(self.prompts['reg'][1]), True]
    if login_cred[2]:
      if self.updateDb(login_cred[:-1]):
        print('Succesfully registered!')
      else:
        print('Couldn\'t register, retry!')
        return None, True
    elif self.checkDb(login_cred[:-1]):
      self.loginid = login_cred[0]
      return random.random(), False
    return None, True
  @abstractmethod
  def gameplay(self):
    pass

In [ ]:
class Player(Blackjack):
  def __init__(self):
    super().__init__()
    if self.session:
      self.hand = None
      self.sum_of_hand = 0
      # self.credit = 1000
  def getHand(self):
    self.hand = list()
    for i in range(2):
      self.draw()
  def draw(self):
    selected = random.choice(self.cards)
    self.cards.remove(selected)
    self.hand.append(selected)
    if selected == (1,11):
      selected = int(input(self.prompts['ace']).strip())
    self.sum_of_hand += selected

  def gameplay(self):
    def pdhand(a, b):
      print(f'Dealer\'s final hand : {a} Dealer\'s score : {b}')
    def display():
      clear_output(True)
      print('Round ', round_no,' Credit : ',self.credit,'Bet : ',bet,'\nyour hand : ', self.hand,' Your score :', self.sum_of_hand,'\nDealer\'s first card : ',dhand[0])
      return
    def dealerDraw(dhand, dcards, dsum):
      card = random.choice(dcards)
      dhand.append(card)
      dcards.remove(card)
      dsum += card if card != (1,11) else (1 if dsum + 11 > self.target else 11)
      return dhand, dcards, dsum
    def dealerInit():
      dcards, dsum, dhand = self.cards, 0, list()
      dhand, dcards, dsum = dealerDraw(dhand, dcards, dsum)
      return dealerDraw(dhand, dcards, dsum)
      return dhand, dcards, dsum
    def dealerHit(dhand, dcards, dsum):
      eq_flag,phand, pcards, psum = False,None,None,None
      while dsum < self.target:
        if dsum > self.sum_of_hand:
          return dhand, dcards, dsum
        elif dsum == self.sum_of_hand:
          eq_flag = True
          phand, pcards, psum = dhand, dcards, dsum
        display()
        dhand, dcards, dsum = dealerDraw(dhand, dcards, dsum)

      if not eq_flag:
        return dhand, dcards, dsum
      else:
        return phand, pcards, psum

    round_no, bet= 1, 0
    dhand, dcards, dsum = dealerInit()
    bet = int(input(self.prompts['game'][0]))
    display()
    while self.credit > 0:
      round_no += 1
      if round_no == 1:
        choice = input(self.prompts['game'][2])
      else:
        choice = input(self.prompts['game'][1])
      if choice in ('s', 'S'):
        dhand, dcards, dsum = dealerHit(dhand, dcards, dsum)
        display()
        if dsum > self.target or self.sum_of_hand > dsum:
          self.credit += bet
          display()
          pdhand(dhand, dsum)
          if round_no == 2:
            retry = input(f'Blackjack! You win! {self.credit} remaining.Play again?(y/n) : ')
          else:
            retry = input(f'Dealer bust! You win! {self.credit} remaining.Play again?(y/n) : ')
        elif dsum == self.sum_of_hand:
          display()
          pdhand(dhand, dsum)
          retry = input(f'Push ! {self.credit} remaining.Play again?(y/n) : ')
        else:
          self.credit -= bet
          display()
          pdhand(dhand, dsum)
          retry = input(f'You bust! {self.credit} remaining. Play again?(y/n) : ')
        if retry in ('y', 'Y'):
          return True
        else:
          break
        display()
      elif choice in ('h', 'H'):
        self.draw()
        if self.sum_of_hand > self.target:
          self.credit -= bet
          display()
          pdhand(dhand, dsum)
          retry = input(f'You bust! {self.credit} remaining. Play again?(y/n) : ')
          if retry in ('y', 'Y'):
            return True
          else:
            break
        display()
      elif choice in ('d', 'D'):
        bet *= 2
        self.draw()
        if self.sum_of_hand > self.target:
          self.credit -= bet
          display()
          pdhand(dhand, dsum)
          retry = input(f'You bust! {self.credit} remaining. Play again?(y/n) : ')
          if retry in ('y', 'Y'):
            return True
          else:
            break
        display()
    if self.credit < 0:
      raise(BrokeException)
    return False

    return
  def startSession(self):
    while True:
      self.sum_of_hand = 0
      self.cards = [(1, 11), 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]
      dhand = self.cards
      self.getHand()
      try:
        if not self.gameplay():
          self.updateCredit()
          # print(self.db)
          self.upload_to_db()
          break
      except BrokeException as e:
        print('e')
        return
    return





In [ ]:
def main():
  retry = False
  while True:
    if retry:
      quit = input('Quit?(y/n) :')
      if quit in ('y', 'Y'):
        break
    instance = Player()
    choice = input('play?(y/n)')
    if choice == 'y' and instance:
      instance.startSession()
      # print(instance.session,pre)
      # pre = instance.session
      del instance
      retry = True
    else:
      break
  return

In [ ]:
main()